In [2]:
import pandas as pd
import numpy as np
import re
from sqlalchemy import func, create_engine  
from utils import * 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
df=pd.read_csv('parsed.csv')
df

,ROW,ID,TEXT,CODE,PKEY,SORT,PLT
0,1,CQ1,GOV Domain Registration Systems,GOV_DOMAINYSTEM,5000,1,400
1,2,CQ1,Access to Data systems,ACCESS_DATA_SYS,5001,2,400
2,3,CQ1,Agency support tool,AGENC_SUPPORT_T,5002,3,400
3,4,CQ1,Business Intelligence Services,BUSI_INTELLIG_S,5003,4,400
4,5,CQ1,Collaboration Tools (SharePoint),COLLABOR_TPOINT,5004,5,400
...,...,...,...,...,...,...,...
483,484,CQ32F,No,N_2_NO,6221,2,482
484,485,CQ32F,Unknown,N_3_UNKNOWN,6222,3,482
485,486,CQ32G,Transmit,N_1_TRANSMIT,6232,1,483
486,487,CQ32G,Receive,N_2_RECEIV,6233,2,483


In [ ]:
def tokenize(s): 
    s = [v for v in s]  
    s = re.sub('[^A-Za-z0-9\s]','', ' '.join(s)).split() 
    return s

In [117]:
df = sql_todf("select * from vwPicklists WHERE UsageField='LEGCHARS' ", connstr)
x = [v for v in df['DisplayValue'].values]  
x = re.sub('[^A-Za-z0-9\s]','', ' '.join(x)).split()
x

['Legacy',
 'codebase',
 'COBOL',
 'MUMPS',
 'Lack',
 'of',
 'or',
 'poor',
 'documentation',
 'Legacy',
 'infrastructure',
 'Adhoc',
 'infrastructure',
 'Hardware',
 'or',
 'software',
 'with',
 'no',
 'vendormanufacturer',
 'support',
 'Outdated',
 'or',
 'inaccurate',
 'inventory',
 'Technology',
 'Type',
 'System',
 'Last',
 'Modernized',
 'Limited',
 'Functionality',
 'Other']

In [81]:
y="""Legacy codebase (COBOL MUMPS)
Lack of or poor documentation
Legacy infrastructure
Ad-hoc infrastructure 
Hardware or software with no vendor/manufacturer support
Outdated or inaccurate inventory
Limited Functionality
other
""".upper().split('\n')
y


['LEGACY CODEBASE (COBOL MUMPS)',
 'LACK OF OR POOR DOCUMENTATION',
 'LEGACY INFRASTRUCTURE',
 'AD-HOC INFRASTRUCTURE ',
 'HARDWARE OR SOFTWARE WITH NO VENDOR/MANUFACTURER SUPPORT',
 'OUTDATED OR INACCURATE INVENTORY',
 'LIMITED FUNCTIONALITY',
 'OTHER',
 '']

In [82]:
jaccard_similarity(x,y)

0.5833333333333334

In [75]:
def jaccard_similarity(x,y): 
  intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
  union_cardinality = len(set.union(*[set(x), set(y)]))
  return intersection_cardinality/float(union_cardinality)

In [63]:
df = pd.DataFrame(columns=["ID","DESCRIPTION"], data=np.matrix([[10,"Cancel ASN WMS Cancel ASN"],
                                                                [11,"MAXPREDO Validation is corect"],
                                                                [12,"Move to QC"],
                                                                [13,"Cancel ASN WMS Cancel ASN"],
                                                                [14,"MAXPREDO Validation is right"],
                                                                [15,"Verify files are sent every hours for this interface from Optima"],
                                                                [16,"MAXPREDO Validation are correct"],
                                                                [17,"Move to QC"],
                                                                [18,"Verify files are not sent"]
                                                                ]))

corpus = list(df["DESCRIPTION"].values)

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

threshold = 0.4

for x in range(0,X.shape[0]):
  for y in range(x,X.shape[0]):
    if(x!=y):
      if(cosine_similarity(X[x],X[y])>threshold):
        print(df["ID"][x],":",corpus[x])
        print(df["ID"][y],":",corpus[y])
        print("Cosine similarity:",cosine_similarity(X[x],X[y]))
        print()

10 : Cancel ASN WMS Cancel ASN
13 : Cancel ASN WMS Cancel ASN
Cosine similarity: [[1.]]

11 : MAXPREDO Validation is corect
14 : MAXPREDO Validation is right
Cosine similarity: [[0.64183024]]

12 : Move to QC
17 : Move to QC
Cosine similarity: [[1.]]

15 : Verify files are sent every hours for this interface from Optima
18 : Verify files are not sent
Cosine similarity: [[0.44897995]]



In [28]:

def sql_todf(query,connstr):
    df=pd.DataFrame() 
    engine = create_engine(connstr) 
    conn = engine.connect() 
    #try: 
    df = pd.read_sql(query,con=conn) 
    #finally: 
    conn.close()
    return df

['LEGACY',
 'CODEBASE',
 '(COBOL',
 'MUMPS)',
 'LACK',
 'OF',
 'OR',
 'POOR',
 'DOCUMENTATION',
 'LEGACY',
 'INFRASTRUCTURE',
 'AD-HOC',
 'INFRASTRUCTURE',
 '',
 'HARDWARE',
 'OR',
 'SOFTWARE',
 'WITH',
 'NO',
 'VENDOR/MANUFACTURER',
 'SUPPORT',
 'OUTDATED',
 'OR',
 'INACCURATE',
 'INVENTORY',
 'TECHNOLOGY',
 'TYPE',
 '',
 'SYSTEM',
 'LAST',
 'MODERNIZED',
 'LIMITED',
 'FUNCTIONALITY',
 'OTHER',
 '']

In [58]:
y="""Legacy codebase (COBOL MUMPS)
Lack of or poor documentation
Legacy infrastructure
Ad-hoc infrastructure 
Hardware or software with no vendor/manufacturer support
Outdated or inaccurate inventory
Limited Functionality
""".replace('\n',' ').upper().split(' ')
y

['LEGACY',
 'CODEBASE',
 '(COBOL',
 'MUMPS)',
 'LACK',
 'OF',
 'OR',
 'POOR',
 'DOCUMENTATION',
 'LEGACY',
 'INFRASTRUCTURE',
 'AD-HOC',
 'INFRASTRUCTURE',
 '',
 'HARDWARE',
 'OR',
 'SOFTWARE',
 'WITH',
 'NO',
 'VENDOR/MANUFACTURER',
 'SUPPORT',
 'OUTDATED',
 'OR',
 'INACCURATE',
 'INVENTORY',
 'LIMITED',
 'FUNCTIONALITY',
 '']

In [59]:
x=lst

jaccard_similarity(x,y)

0.7931034482758621